In [1]:
"""
This module is coded entirely by Oscar Serna.
This code makes use of the code written by Jimmy Song in the book Programming Bitcoin.
This code only tries to build on Jimmi Song's code and take it to the next step as 
suggested by him in chapter 14 of his book.

This is just an educational-purpose code. The author does not take any responsibility
on any losses caused by the use of this code.
"""

from ecc import PrivateKey, S256Point, Signature
from script import p2pkh_script, p2sh_script, Script, p2wpkh_script
from helper import decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160, h160_to_p2sh_address, encode_varint
from tx import TxIn, TxOut, Tx, TxFetcher
import hashlib

class Account():
    
    """
    To create account use from_phrase() method.
    """
    
    def __init__(self, _privkey):
        """
        Initialize the account with a private key in Integer form.
        """
        self.privkey = PrivateKey(_privkey)
        self.testnet_address = self.privkey.point.address(testnet = True)
        self.address = self.privkey.point.address(testnet = False)
        
        #p2spkh script: OP_DUP, OP_HASH160, <20-BYTE HASH>, OP_EQUALVERIFY, OP_CHECKSIG
        
        self.redeem_script_segwit = p2wpkh_script(self.privkey.point.hash160())
        serialized_redeem = self.redeem_script_segwit.raw_serialize()
        self.segwit_address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=False)
        self.testnet_segwit_address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=True)
        
    def __repr__(self):
        return f"Private Key Hex: {self.privkey.hex()}"
        
    
    @classmethod
    def from_phrase(cls,phrase, endian="big"):
        """
        phrase must be in bytes.
        endian can be either "big" or "little". Change later to boolean "bignendian"
        Returns an account using the phrase chosen which is converted to an Integer
        """
        if endian not in ["big","little"]:
            raise Exception( f'endian can be either "big" or "little" not "{endian}"')
            
        if isinstance(phrase, str):
            phrase = phrase.encode('utf-8')
            
        if isinstance(phrase, bytes):
            return cls(int.from_bytes(phrase,endian))
        
        else:
            raise Exception( f"The phrase must be a string or bytes, not {type(phrase)}" )


class MultSigAccount():
    
    def __init__(self,m, n,  _privkeys, segwit=True):
        """
        Initialize the account with a private key in Integer form.
        m: the minumin amount of signatures required to spend the money.
        n: total amount of signatures that can be used to sign transactions.
        Note: Therefore:
        a- m has to be less or equal to n.
        b- n has to be equal to the length of the array _privkeys
        Also:
        c- n could be 20 or less, but to keep simplicity in the code, n can only be 16 or less.
        """
        if n != len(_privkeys):
            raise Exception("n must be equal to the amount of private keys")
        if m < 1 or m > 16 or n < 1 or n > 16:
            raise Exception("m and n must be between 1 and 16")
        if m > n:
            raise Exception("m must be always less or equal than n")
            
            
        self.privkeys = [PrivateKey(privkey) for privkey in _privkeys]
        self.m = m
        self.n = n
        pubkeys = [x.point.sec() for x in self.privkeys]
        self.redeem_script = Script([m+80, *pubkeys, n + 80, 174])
        serialized_redeem = self.redeem_script.raw_serialize()
        self.testnet_address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=True)
        self.address = h160_to_p2sh_address(hash160(serialized_redeem), testnet=False)
        
        
    def __repr__(self):
        return f"Private Key Hex: {self.privkey.hex()}"
        
    
    @classmethod
    def from_phrases(cls, m , n , phrases):
        """
        phrase: must be a list of tuples of (bytes, endian) or (string, endian). i.e:
            [(b"my secret","little"),("my other secret","big")]
            
        endian: can be either "big" or "little"
        Returns a multisignature account using the phrases chosen which are converted to Integers
        """
        keys = [int.from_bytes(key[0],key[1]) for key in phrases]
        return cls(m,n,keys)
       
        
class Build_TX():
    @classmethod
    def get_index(cls,outs_list, address):
        """
        Supporting method.
        receives the list of outputs from transaction and find the index of 
        particular output of interest.
        outs_list: is the list of outputs of previous transaction where the UTXO is.
        address: the address trying to spend the UTXO.
        """
        print(f"Address: {address}")
        print(f"Address: {decode_base58(address)}")
        for index,out in enumerate(outs_list):
            print(out.script_pubkey.cmds[2])
            #print(out.script_pubkey.cmds[1])
            #print(out.script_pubkey.cmds[0])
            #print(out.script_pubkey.cmds[3])
            #print(out.script_pubkey.cmds[4])
            if out.script_pubkey.cmds[2] == decode_base58(address) or out.script_pubkey.cmds[1] == decode_base58(address):
                return index
        raise Exception( "output index not found")

    @classmethod
    def get_amount_utxo(cls,outs_list, index):
        """
        Supporting method.
        receives the list of outputs from transaction and the index of 
        particular output of interest and returns the amount of the UTXO.
        outs_list: is the list of outputs of previous transaction where the UTXO is.
        index: the index of the UTXO in the list of all the outputs.
        """
        return outs_list[index].amount

    @classmethod
    def get_tx_ins_utxo(cls,prev_tx_id_list, receiving_address, testnet=True):
        """
        Receives a list of transaction ids where the UTXOs to spend are, and 
        also the receiving address to return a valid tx_in list to create
        a transaction.
        prev_tx_id_list: list of the transaction ids where the UTXOs are.
        receiving_address: the address trying to spend the UTXO (String).
        testnet: if the transaction is in testnet or not (boolean).
        """
        tx_ins = []

        for prev_tx_id in prev_tx_id_list:
            prev_tx = TxFetcher.fetch(prev_tx_id, testnet)
            prev_index = cls.get_index(prev_tx.tx_outs, receiving_address)
            tx_in = TxIn(bytes.fromhex(prev_tx_id),prev_index)
            utxo = cls.get_amount_utxo(prev_tx.tx_outs, prev_index)
            #print(f"index 1: {prev_index}, amount: {utxo}")
            tx_ins.append({"tx_in": tx_in, "utxo": utxo})

        return tx_ins

    @classmethod
    def calculate_fee(cls,version, tx_ins, tx_outs, locktime, privkey, redeem_script, 
                      testnet=True, multisig =False, fee_per_byte = 8, segwit = False ):
        """
        privkey: can be just one or a list of private keys in the case of multisignature.
        """
        my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
        print(my_tx)

        # sign the inputs in the transaction object using the private key
        if multisig:
            for tx_input in range(len(tx_ins)):
                print(my_multisig_tx.sign_input_multisig(tx_input, privkey, redeem_script))
        else:
            for tx_input in range(len(tx_ins)):
                print(my_tx.sign_input(tx_input, privkey, segwit = segwit))
            # print the transaction's serialization in hex

        #Let's calculate the fee and the change:
        tx_size = len(my_tx.serialize().hex())
        #fee_per_byte = 8 # I changed from 2 to 10 after sending this transaction because it had really low appeal to miners.
        fee = tx_size * fee_per_byte
        print(f"fee: {fee}")
        return fee
    
    @classmethod
    def calculate_change(cls, utxo_list, fee, amountTx):
        """
        utxo_list: the list of the amounts of every utxo.
        amountTx: the list of the ammounts of every transaction output.
        fee: the fee of the transaction.
        Returns the respective amount of the change.
        """
        total_utxo = sum(utxo_list)
        total_out = sum(amountTx)
        change = total_utxo - fee - total_out
        print(f"change {change}")
        if change < 0:
            raise Exception( f"Not enough utxos: total_utxo {total_utxo} and total_out {total_out} meaning change = {change}")
        #Let's make sure that we are actually spending the exact amount of the UTXO
        total_send=fee+total_out+change
        diff = total_utxo-total_send
        print(f"total {total_send}, diff: {diff}")

        return change

    @classmethod
    def build_tx(cls, utxo_tx_id_list, outputAddress_amount_list, account, testnet = True, fee=None, segwit=False,witness_address = False):
        """
        utxo_tx_id_list: the list of the transaction ids where the UTXOs are.
        outputAddress_amount_list: a list of tuples (to_address:amount) specifying
        the amount to send to each address.
        account: must be an Account object.
        If fee is specifyed, then the custom fee will be applied.
        
        Returns the hex of the raw transaction.
        """
        print(f"Sender: \nmainnet: {account.address}\ntestnet: {account.testnet_address}")
        #Validation process:
        if testnet:
            for addr in outputAddress_amount_list:
                if addr[0][0] not in "2mn":
                    raise Exception (f"{addr[0]} not a testnet address. Funds will be lost!")
                if addr[1] < 1:
                    raise Exception (f"{addr[1]} not a valid amount. It should be greater than 1 satoshis")
        else:
            for addr in outputAddress_amount_list:
                if addr[0][0] not in "13":
                    raise Exception (f"{addr[0]} not a mainnet bitcoin address. Funds will be lost!")
                if addr[1] < 1:
                    raise Exception (f"{addr[1]} not a valid amount. It should be greater than 1 satoshis")
              
        # initializing variables
        tx_outs =[]
        tx_ins =[]
        change = int(0.01 * 100000000)# we are going to fix this later
        #amountTx= int(0.01 * 100000000)
        
        #https://en.bitcoin.it/wiki/List_of_address_prefixes
        #We create the tx outputs based on the kind of address (multisig or normal)
        for output in outputAddress_amount_list:
            #For normal p2pkh
            if output[0][0] in "1mn" :
                tx_outs.append( TxOut(output[1], p2pkh_script(decode_base58(output[0]))))
            #For multidignature p2sh
            elif output[0][0] in "23" :
                tx_outs.append( TxOut(output[1], p2sh_script(decode_base58(output[0]))))
        
        #Let's return the fake change to our same address. 
        #This will change later when BIP32 is implemented.
        if testnet:
            if account.testnet_address[0] in "mn":
                tx_outs.append( TxOut(change, p2pkh_script(decode_base58(account.testnet_address))))
            elif account.testnet_address[0] == "2":
                tx_outs.append( TxOut(change, p2sh_script(decode_base58(account.testnet_address))))
        else:
            if account.address[0] == "1":
                tx_outs.append( TxOut(change, p2pkh_script(decode_base58(account.address))))
            elif account.address[0] == "3":
                tx_outs.append( TxOut(change, p2sh_script(decode_base58(account.address))))
                
        #Creating the Tx_In list:
        if testnet:
            if witness_address:
                tx_ins_utxo = cls.get_tx_ins_utxo(utxo_tx_id_list, account.testnet_segwit_address, testnet)
            else:
                tx_ins_utxo = cls.get_tx_ins_utxo(utxo_tx_id_list, account.testnet_address, testnet)
        else: 
            if witness_address:
                tx_ins_utxo = cls.get_tx_ins_utxo(utxo_tx_id_list, account.segwit_address, testnet)
            else:    
                tx_ins_utxo = cls.get_tx_ins_utxo(utxo_tx_id_list, account.address, testnet)
            
        tx_ins = [x["tx_in"] for x in tx_ins_utxo]
        utxos = [x["utxo"] for x in tx_ins_utxo]
        
        #CREATING THE TRANSACTION RIGHT HERE:
        my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=testnet, segwit=segwit)
        
        fee = cls.calculate_fee(1, tx_ins, tx_outs, 0, privkey=account.privkey, redeem_script=None, testnet=True, segwit = segwit)
        change = cls.calculate_change(utxos, fee, [x[1] for x in outputAddress_amount_list])
        
        my_tx.tx_outs[-1].amount = change
        
        #for tx_input in range(len(tx_ins)):
        #    print(f"SEGWIT: {segwit}")
        #    if not my_tx.sign_input(tx_input, account.privkey, segwit = segwit):
        #        raise Exception("Signature failed")
        if multisig:
            for tx_input in range(len(tx_ins)):
                print(my_multisig_tx.sign_input_multisig(tx_input, privkey, redeem_script))
        else:
            for tx_input in range(len(tx_ins)):
                print(my_tx.sign_input(tx_input, privkey, segwit = segwit))
            # print the transaction's serialization in hex
        
        return my_tx
        

In [2]:
account1 = Account.from_phrase(b"Oscar Eduardo Serna Rosero","big")

segwitAccount = Account.from_phrase(b"Oscar Eduardo Serna Rosero","big")

account1.testnet_address

'mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8'

In [3]:
multisig_acc = MultSigAccount.from_phrases(2,2,[(b"Oscar Eduardo Serna Rosero","big"), (b"Oscar Eduardo Serna Rosero","little")])

In [4]:
prev_tx_id_list = ["4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c8410952241a4d"]
my_tx = Build_TX.build_tx(prev_tx_id_list, [(segwitAccount.testnet_segwit_address, 1000000)], account1, True)

Sender: 
mainnet: 18XZD7yQzQrc8Wka1B3QevTT9uxXWhcMTZ
testnet: mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8
length script: 106
current_byte : 71
current_byte : 33
length script: 106
current_byte : 71
current_byte : 33
length script: 23
current_byte : 169
op_code : 169
current_byte : 20
current_byte : 135
op_code : 135
length script: 25
current_byte : 118
op_code : 118
current_byte : 169
op_code : 169
current_byte : 20
current_byte : 136
op_code : 136
current_byte : 172
op_code : 172
816aa0bc346ae4622b905a4b6aa1223ce8c09047430185c464132acdb5075357:0
Address: mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8
Address: b'R\x90>\xfc\x10\x04\xde\x01\x88;\xa3h{\xe2\xa8\xeaOk\x1b\x19'
135
b'R\x90>\xfc\x10\x04\xde\x01\x88;\xa3h{\xe2\xa8\xeaOk\x1b\x19'
4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c8410952241a4d:1
4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c8410952241a4d:1
tx: 8b44cb288015e2d5f182212d7444af3118f863ce54394ec58a9b1e12ecd8be49
version: 1
tx_ins:
4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c

In [5]:
my_tx

tx: c09a9e1ea1861fff3aa5d2114e7e8d3c3fcce5b9ff91d126da111bd08e127339
version: 1
tx_ins:
4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c8410952241a4d:1
tx_outs:
1000000:OP_HASH160 f7c07f67fb6e54ea6264b1a508b24e34ec999c56 OP_EQUAL
953042:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0

In [6]:
segwitAccount.testnet_segwit_address

'2NFqDVe386fZ6AeQaox5ZK8qkSm3CCWntVT'

In [7]:
segwitAccount.testnet_segwit_address

'2NFqDVe386fZ6AeQaox5ZK8qkSm3CCWntVT'

In [8]:
prev_tx_id_list = ["c09a9e1ea1861fff3aa5d2114e7e8d3c3fcce5b9ff91d126da111bd08e127339"]
my_tx = Build_TX.build_tx(prev_tx_id_list, [(account1.testnet_address, 50000)], segwitAccount, True, segwit = True,witness_address = True)

Sender: 
mainnet: 18XZD7yQzQrc8Wka1B3QevTT9uxXWhcMTZ
testnet: mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8
length script: 106
current_byte : 71
current_byte : 33
length script: 23
current_byte : 169
op_code : 169
current_byte : 20
current_byte : 135
op_code : 135
length script: 25
current_byte : 118
op_code : 118
current_byte : 169
op_code : 169
current_byte : 20
current_byte : 136
op_code : 136
current_byte : 172
op_code : 172
4aed764c2cc2cc2a6bb0870e6b7b5560918bc8bf79bd4f44e0c8410952241a4d:1
Address: 2NFqDVe386fZ6AeQaox5ZK8qkSm3CCWntVT
Address: b'\xf7\xc0\x7fg\xfbnT\xeabd\xb1\xa5\x08\xb2N4\xec\x99\x9cV'
135
c09a9e1ea1861fff3aa5d2114e7e8d3c3fcce5b9ff91d126da111bd08e127339:0
c09a9e1ea1861fff3aa5d2114e7e8d3c3fcce5b9ff91d126da111bd08e127339:0
tx: 0effab73d4ee7c0a034ede0a201380543e7d13262206b5853aeee22b2c7ba1a5
version: 1
tx_ins:
c09a9e1ea1861fff3aa5d2114e7e8d3c3fcce5b9ff91d126da111bd08e127339:0
tx_outs:
50000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
100

In [9]:
my_tx.serialize().hex()

'010000000001013973128ed01b11da26d191ffb9e5cc3f3c8d7e4e11d2a53aff1f86a11e9e9ac0000000001716001452903efc1004de01883ba3687be2a8ea4f6b1b19ffffffff0250c30000000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac10760e00000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac02483045022100a4b78d2f235f85dcfd0ac029d2d1634c4d65d0ec776265a6874eac666e6ff6e00220559c136c049298632f5399a7f68d9016e9e25a4857697b07655000ba5345aeb2012103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e400000000'

In [10]:
from script import Script
from io import BytesIO
from helper import encode_varint

In [11]:


RedScript = Script.parse(BytesIO(b"52903efc1004de01883ba3687be2a8ea4f6b1b19"))

length script: 53
current_byte : 50


IndexError: index out of range

In [ ]:
RedScript.cmds

In [ ]:
len(b"!\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4")

In [ ]:
segwitAccount.redeem_script_segwit

In [ ]:
encode_varint(20)

In [ ]:
hex(int.from_bytes(my_tx.tx_ins[0].witness,"big"))

In [ ]:
my_tx.tx_ins[0].witness